# De l’étiquetage avec *TreeTagger*

## Un étiqueteur pré-entraîné

*TreeTagger* est un étiqueteur morpho-syntaxique pré-entraîné pour de multiples langues (anglais, allemand, français, espagnol, italien…). Il est développé à [l’IMS de Stuttgart](https://www.ims.uni-stuttgart.de/) par Helmut Schmid. Sans être le meilleur, il est toutefois toujours plébiscité pour des raisons historiques, pour sa rapidité et pour sa couverture linguistique. Parmi les reproches qui persistent pour sa partie française, nous retenons l’absence d’information sur le corpus d’apprentissage ou sur le lexique utilisé.

La procédure d’installation de l’utilitaire et de ses fichiers de paramètres est détaillée dans [la documentation en ligne](https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/). Vous trouverez ceci dit les fichiers nécessaires dans le répertoire *TreeTagger*.

## Segmentation d’un texte

En entrée, *TreeTagger* accepte une liste de tokens afin de calculer les probabilités des étiquettes à leur assigner. Un tokénisateur basique est fourni avec la distribution :

In [ ]:
! echo "A Lannister always pays his debts." | perl ./TreeTagger/utf8-tokenize.perl

L’algorithme par défaut est prévu pour une segmentation sur l’espace. Des options permettent d’activer des comportements différenciés pour certaines langues :

- `-e` : pour l’anglais ;
- `-f` : pour le français ;
- `-i` : pour l’italien ;
- `-z` : pour le galicien ;
- `-a` : pour fournir un fichier personnalisé d’abréviations.

In [ ]:
! echo "C’est aujourd’hui ton rendez-vous chez l’ostéopathe ?" | perl ./TreeTagger/utf8-tokenize.perl -f

Le script a correctement considéré *aujourd’hui* comme un seul mot quand *rendez-vous* a été séparé en deux. Essayons de lui transmettre un fichier pour gérer certaines spécificités du français :

In [ ]:
! echo "C’est aujourd’hui ton rendez-vous chez l’ostéopathe ?" | perl ./TreeTagger/utf8-tokenize.perl -f -a ./TreeTagger/french-abbreviations

Le fichier *french-abbreviations* contient une simple liste de mots constituée en connaissance de cause, c’est-à-dire après analyse des erreurs résiduelles du tokénisateur de *TreeTagger*.

## Annotation des tokens

### Étiquetage en parties du discours

La procédure d’étiquetage de *TreeTagger* implémente un modèle probabiliste qui, pour des raisons de performance, prend la forme d’un arbre de décision. Différents modèles de langage sont disponibles sur la page des téléchargements. Il s’agit de fichiers de paramètres qui portent une extension en *.par* :

In [ ]:
! echo "A Lannister always pays his debts." | perl ./TreeTagger/utf8-tokenize.perl -e | ./TreeTagger/tree-tagger ./TreeTagger/english.par

L’ajout d’une option `-token` permet de révéler le token étiqueté :

In [ ]:
! echo "A Lannister always pays his debts." | perl ./TreeTagger/utf8-tokenize.perl -e | ./TreeTagger/tree-tagger -token ./TreeTagger/english.par

Les étiquettes attribuées dépendent du modèle de langage indiqué :

In [ ]:
! echo "A Lannister always pays his debts." | perl ./TreeTagger/utf8-tokenize.perl -f | ./TreeTagger/tree-tagger -token ./TreeTagger/french.par

*TreeTagger* ne vérifie donc pas la vraisemblance des tokens en entrée avec le modèle de langage spécifié. La pertinence de certains résultats peut s’en ressentir !

**Remarque :** pour prendre connaissance de la signification des étiquettes, se référer à la documentation en ligne.

### Lemmatisation

Par lemmatisation, on entend l’opération consistant à obtenir la forme canonique d’un mot. Par exemple, *parent* serait le lemme de *parents*, ou *chanter* celui de la forme fléchie *chantes*.

L’option `-lemma` active la fonction :

In [ ]:
! echo "A Lannister always pays his debts." | perl ./TreeTagger/utf8-tokenize.perl -e | ./TreeTagger/tree-tagger -token -lemma ./TreeTagger/english.par

Pour *TreeTagger*, cette étape n‘est résolue que par extraction depuis un lexique où à chaque lemme est associé une étiquette. En conséquence, si dans la phrase suivante il parvient bien à désambiguiser les occurrences du mot *dérive*, c’est uniquement grâce à leurs étiquettes :

In [ ]:
! echo "Le navire dérive à cause d’une dérive instable." | perl ./TreeTagger/utf8-tokenize.perl -f | ./TreeTagger/tree-tagger -token -lemma ./TreeTagger/french.par

*TreeTagger* ne fait aucune hypothèse quant aux lemmes aussi, en cas de doute, il liste les différentes possibilités avec une barre verticale :

In [ ]:
! echo "Il a été secouru par le fils du jardinier." | perl ./TreeTagger/utf8-tokenize.perl -f | ./TreeTagger/tree-tagger -token -lemma ./TreeTagger/french.par

Étiqueté comme *NOM*, le mot *fils* peut aussi bien être le pluriel de *fil* – comme dans *le fil de l’histoire* ou *un fil de lin* – que l’une des formes du lemme *fils*.

## Frontières de décision

Considérant les ambiguïtés inhérentes aux langues et le modèle probabiliste derrière *TreeTagger*, rien de surprenant à ce qu’il effectue un choix au regard du contexte.

Prenons un énoncé qui ne pose guère de problème :

In [ ]:
! echo "Le capitaine Grant a mené le bataillon au combat." | perl ./TreeTagger/utf8-tokenize.perl -f | ./TreeTagger/tree-tagger -token -lemma ./TreeTagger/french.par

Ici, bien qu’inconnu, *Grant* est bien considéré comme un nom propre (*NAM*) grâce à la présence de la majuscule. Retirons à *TreeTagger* cette faculté de s’appuyer sur le préfixe avec l’option `-ignore-prefix` :

In [ ]:
! echo "Le capitaine Grant a mené le bataillon au combat." | perl ./TreeTagger/utf8-tokenize.perl -f | ./TreeTagger/tree-tagger -token -ignore-prefix ./TreeTagger/french.par

*Grant* est maintenant considéré comme un adjectif, un choix logique quand on le sait précédé par un nom.

Pour comprendre le mécanisme, il nous faut révéler les frontières de décision avec les options `-threshold` et `-prob`. Le seuil est défini dans l’intervalle $]0,1]$ :

In [ ]:
! echo "Le capitaine Grant a mené le bataillon au combat." | perl ./TreeTagger/utf8-tokenize.perl -f | ./TreeTagger/tree-tagger -token -ignore-prefix -threshold 0.01 -prob ./TreeTagger/french.par

Pour la majorité des mots de la phrase, *TreeTagger* n’a aucun doute. Il est en revanche moins assuré pour le fameux *Grant* qu’il identifie d’ailleurs à 71 % comme un participe présent (*VER:ppre*) à cause de sa terminaison en *-ant*.

Pourquoi, alors, choisit-il au final de l’étiqueter *ADJ* ? Les probabilités affichées sont en fait celles du token considéré sans le contexte.

In [ ]:
! echo "Le capitaine Grant a mené le bataillon au combat." | perl ./TreeTagger/utf8-tokenize.perl -f | ./TreeTagger/tree-tagger -token -ignore-prefix -threshold 0.01 -prob ./TreeTagger/french.par

Promouvons notre capitaine au rang de général :

In [ ]:
! echo "Le général Grant a mené le bataillon au combat." | perl ./TreeTagger/utf8-tokenize.perl -f | ./TreeTagger/tree-tagger -token -threshold 0.01 -prob ./TreeTagger/french.par

Le mot *général* devient pour lui un adjectif, ce qui met en balance l’étiquette de *Grant*. Au final, même en lui autorisant l’appui du préfixe, *TreeTagger* fait également le mauvais choix pour *Grant* :

In [ ]:
! echo "Le général Grant a mené le bataillon au combat." | perl ./TreeTagger/utf8-tokenize.perl -f | ./TreeTagger/tree-tagger -token ./TreeTagger/french.par

## Améliorer un modèle de langage

Nous l’avons vu, *TreeTagger* n’est pas infaillible. Heureusement, plusieurs pistes existent :

- choisir un autre modèle de langage ;
- apporter des corrections aux ressources à l’origine du modèle de langage.

### Évaluer un modèle de langage

Dans le dossier *data* se trouve une version étiquetée du poème *Le dormeur du Val* de Rimbaud, sous le nom *dormeur_du_val_tagged.tsv*. Elle a été obtenue après application du modèle de langage par défaut pour le français.

À titre de comparaison, utilisons plutôt un modèle de langage prévu pour le français parlé, [Perceo](https://www.ortolang.fr/market/corpora/perceo) :

In [ ]:
! cat ./data/dormeur_du_val.txt | perl ./TreeTagger/utf8-tokenize.perl -f | ./TreeTagger/tree-tagger ./TreeTagger/spoken-french.par > ./data/dormeur_du_val_tagged_spoken.tsv

Et fusionnons les deux étiquetages dans un nouveau fichier *dormeur_du_val_eval.tsv*, constitué de trois colonnes :

- le token ;
- l’étiquetage manuel ;
- l’étiquetage automatique.

In [ ]:
! paste -d $'\t' <(cut -d $'\t' -f 1,2 ./data/dormeur_du_val_tagged.tsv) ./data/dormeur_du_val_tagged_spoken.tsv > ./data/dormeur_du_val_eval.tsv

L’objectif maintenant est de vérifier combien de fois les deux dernières colonnes diffèrent :

In [ ]:
! awk -F '\t' '$2 != $3 {print $0}' ./data/dormeur_du_val_eval.tsv | wc -l

Sur un total de 145 tokens, cela nous donne une exactitude de 56,55 %, autant dire très peu. Toutefois, si l’on regarde de plus près, la plupart des erreurs sont dues à une différence dans l’écriture des étiquettes. Même si Perceo s’appuie en partie sur celles de *TreeTagger*, il en intègre de nouvelles et en écrit certaines avec des minuscules là où auparavant il y avait des majuscules (ex. : *PRO:rel* pour *PRO:REL*).

### Définir un modèle de langage

Comme les modèles de langage sont décrits dans des fichiers binaires en *.par*, nous n’intervenons pas directement dessus pour leur apporter des modifications mais sur les ressources qui ont permis de les générer. *TreeTagger* est en effet fourni avec un utilitaire `train-tree-tager` pour créer un de ces fichiers *.par* à partir de trois ressources :

- un lexique des formes fléchies ;
- un corpus d’apprentissage ;
- une liste d’étiquettes.

#### Le lexique des formes fléchies

Il s’agit d’un fichier plat où chaque ligne contient un mot-forme suivi d’une tabulation puis d’une paire étiquette-lemme eux-mêmes séparés par une espace. En cas d’ambiguïté sur l’étiquetage, il peut exister plusieurs paires qui seront listées derrière des tabulations, comme dans l’exemple :

```csv
fil    N fil
fils   N fils    N fil
```

#### Le corpus d’apprentissage

À l’aide de ce corpus, *TreeTagger* va apprendre un langage, c’est-à-dire une façon de représenter la langue. Pour cela, il va se reposer sur un unique fichier tabulé constitué des mots-formes et de leur étiquette associée, sans oublier la ponctuation. À l’évidence, l’étiquetage doit être irréprochable.

#### La liste des étiquettes

Dans ce fichier, il suffit simplement de lister les étiquettes séparées par une espace. Lorsque *TreeTagger* sera face à un mot pour lequel il ne trouve aucun lemme, il cherchera parmi cette liste l’étiquette à lui affecter.

#### Entraîner *TreeTagger*

Dans le répertoire *data*, trois fichiers très simples vont servir à calculer un modèle de langages :

- *train-lexicon.txt*
- *train-corpus.txt*
- *train-tags.txt*

La commande suivante va se charger de générer un fichier *basic-model.par* :

In [ ]:
! ./TreeTagger/train-tree-tagger -utf8 ./data/train-lexicon.txt ./data/train-tags.txt ./data/train-corpus.txt ./TreeTagger/basic-model.par

Testons notre modèle avec une phrase qui fait partie du corpus d’apprentissage :

In [ ]:
! echo "Le petit chien boit de l’eau." | perl ./TreeTagger/utf8-tokenize.perl -f | ./TreeTagger/tree-tagger -token -lemma ./TreeTagger/basic-model.par

Et maintenant avec une phrase légèrement différente :

In [ ]:
! echo "Jean aime boire de la bière." | perl ./TreeTagger/utf8-tokenize.perl -f | ./TreeTagger/tree-tagger -token -lemma ./TreeTagger/basic-model.par

Un résultat guère encourageant, mais tout à fait compréhensible au regard des ressources que nous lui avons fournie ! À noter que la commande `train-tree-tagger` vient également avec des options qui peuvent améliorer l’apprentissage, dont :

- `-st` : l’étiquette qui sert à baliser la fin d’une phrase et généralement associée à des signes de ponctuation forts comme le point, le point d’exclamation ou le point d’interrogation. Par défaut, l’option vaut `SENT`.
- `-cl` : sert à déterminer la fenêtre du contexte. Par défaut, elle est fixée à 2, ce qui correspond à un contexte de trigrammes. Pour des petits corpus d’apprentissage, il est souvent préférable de réduire la fenêtre à 1.
- `-dtg` : lorsque la mesure du gain d’information n’est pas au-dessus de ce seuil, le nœud de l’arbre de décision est supprimé. Il est à 0,7 par défaut.

Pour les autres, consulter la documentation officielle.